In [1]:
import cobra
import cplex
import optlang
import sys
import json
import logging

from configparser import ConfigParser
config = ConfigParser()
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)
import cobrakbase
from cobrakbase.core.kbasefba.newmodeltemplate_builder import NewModelTemplateBuilder
from modelseedpy import MSCommunity,MSATPCorrection,MSPackageManager, FBAHelper, MSBuilder, MSGapfill
from modelseedpy.helpers import get_template
from IPython.display import HTML

cobrakbase 0.2.8


In [2]:
#Now we pull the community model and media out of KBase
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("Cjaponicus_Ecoli_Community",92637)
media = kbase_api.get_from_ws("ChitinM9Media",91858)
#Now we create a community model analysis object in ModelSEEDpy
model.solver = 'optlang-cplex'
comm = MSCommunity(model,["Cjaponicus","Ecoli"],{"Cjaponicus":0.5,"Ecoli":0.5})
comm.lp_filename = "Cjaponicus_Ecoli"
#Now we use a utility to test that each compartment in the model can grow and make ATP
output = comm.test_individual_species(media,allow_interaction=False)
output

,Species,Biomass,ATP
0,Cjaponicus,<Solution 1.945 at 0x7ff1a9b77850>,<Solution 554.318 at 0x7ff1a9bac1d0>
1,Ecoli,<Solution 1.151 at 0x7ff1a9bc5f10>,<Solution 99.375 at 0x7ff1a9bf3550>


In [3]:
#Note the large amount of ATP generated by Cjaponicus - let's use the ATP correction tool to fix this
core = NewModelTemplateBuilder.from_dict(get_template('template_core'), None).build()
#We need to use one of the media in narrative https://narrative.kbase.us/narrative/94026
#These are the medias that work with the ATP correction tool
glucosemedia = kbase_api.get_from_ws("Glc.atp",94026)
#Initialize the ATP correction object selecting the c1 compartment, which corresponds to Cjaponicus
msatp = MSATPCorrection(comm.model,core,[glucosemedia],"c1")
msatp.evaluate_growth_media()
msatp.determine_growth_media()
msatp.apply_growth_media_gapfilling()
msatp.expand_model_to_genome_scale()

Glc.atp = 1.0
Removing rxn10966_c1 <


In [4]:
#So we identified rxn10966_c1 as the reaction causing the problem
#This reaction has been removed from the model in memmory in this notebook, BUT NOT ON KBASE!!
#Let's test that the removal of rxn10966_c1 fixes the problem
output = comm.test_individual_species(media,allow_interaction=False)
output

,Species,Biomass,ATP
0,Cjaponicus,<Solution 0.042 at 0x7ff1a936b9d0>,<Solution 2.500 at 0x7ff1aae58950>
1,Ecoli,<Solution 1.151 at 0x7ff1aae589d0>,<Solution 99.375 at 0x7ff1aae581d0>


In [ ]:
#Success!! Cjaponicus no longer overproduces ATP
#Note - the modified community model cannot yet be saved directly to KBase
#So we use the model editor in KBase to manually remove rxn10966_c1
#In the meantime, let's go ahead and simulate our model growth
#Setting the objective to the default of maximizing community biomass
comm.set_objective()
#Adding capacity constraints (the number is an adjustable parameter)
comm.constrain(None,900,None)
#Running FBA
output = comm.run(media)
print(comm.model.summary())

In [6]:
#Computing interactions from FBA solution and reporting out to a dataframe
output = comm.compute_interactions()
HTML(output.to_html(render_links=True, escape=False))

,IDs,Metabolites/Species,Cjaponicus,Ecoli,Environment
0,cpd00067_e0,H+_e0,-0.466224,16.310086,-15.843862
1,cpd04097_e0,Pb_e0,0.0,0.0,0
2,cpd00012_e0,PPi_e0,0.0,0.0,0
3,cpd01030_e0,Salicin_e0,0.0,0.0,0
4,cpd00129_e0,L-Proline_e0,-0.002406,0.002406,0
5,cpd00971_e0,Na+_e0,0.0,0.0,0.0
6,cpd01012_e0,Cd2+_e0,0.0,0.0,0
7,cpd00635_e0,Cbl_e0,0.0,0.0,0
8,cpd00246_e0,Inosine_e0,0.00022,-0.00022,0
9,cpd00276_e0,GLUM_e0,-0.204946,0.204946,0
